In [29]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from time import sleep
import concurrent.futures
import csv

In [30]:
# 対象の企業リスト
companies = [
    "ゴールドマン・サックス証券", "JPモルガン証券", "メリルリンチ日本証券", "大和証券キャピタル・マーケッツ",
    "東京海上日動火災保険", "日本政策投資銀行", "UBS証券株式会社", "野村證券", "三菱UFJ銀行", "三井住友信託銀行",
    "A.T.カーニー", "ボストン・コンサルティンググループ", "マッキンゼー・アンド・カンパニー・インク・ジャパン", "三菱総合研究所",
    "サントリーホールディングス", "ソニーグループ株式会社", "株式会社日立製作所", "プロクター・アンド・ギャンブル・ジャパン",
    "日本アイ・ビー・エム", "楽天グループ株式会社",
    "三菱商事株式会社", "三井物産株式会社", "三井不動産株式会社", "ソフトバンクグループ"
]

#companies = [    "ゴールドマン・サックス証券", "JPモルガン証券"]

In [31]:
# スクレイピングの結果を格納するための辞書
results = {}

# スクレイピング関数
def fetch_company_data(search_term):
    search_url = 'https://en-hyouban.com/search/'
    params = {'SearchWords': search_term}
    response = requests.get(search_url, params=params)
    if response.ok:
        return response.text
    return None

def find_company_profile_page(html_content, company_name):
    soup = BeautifulSoup(html_content, 'html.parser')
    for a in soup.select('a'):
        if company_name in a.text:
            return urljoin('https://en-hyouban.com', a['href'])
    return None

def fetch_company_profile_page(url):
    response = requests.get(url)
    if response.ok:
        return response.text
    return None

def parse_company_profile_page(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    salary = soup.select_one('.fsize-40.font-change-30.font-roboto.font-weight-bold:nth-of-type(1)').get_text(strip=True) if soup.select_one('.fsize-40.font-change-30.font-roboto.font-weight-bold:nth-of-type(1)') else '情報なし'
    overtime = soup.select_one('.fsize-40.font-change-30.font-roboto.font-weight-bold:nth-of-type(2)').get_text(strip=True) if soup.select_one('.fsize-40.font-change-30.font-roboto.font-weight-bold:nth-of-type(2)') else '情報なし'
    return {
        'salary': salary,
        'overtime': overtime
    }


def parse_company_profile_page(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    salary = soup.select_one('.fsize-40.font-change-30.font-roboto.font-weight-bold:nth-of-type(1)').get_text(strip=True) if soup.select_one('.fsize-40.font-change-30.font-roboto.font-weight-bold:nth-of-type(1)') else '情報なし'
    
    # 使用する警告を修正: 'text' -> 'string'
    overtime_section = soup.find('div', string=lambda t: t and '残業時間' in t)
    if overtime_section:
        overtime = overtime_section.find_next('span', class_='font-change-30').get_text(strip=True) if overtime_section.find_next('span', class_='font-change-30') else '情報なし'
    else:
        overtime = '情報なし'

    return {
        'salary': salary,
        'overtime': overtime
    }


def parse_company_profile_page(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    salary = soup.select_one('.fsize-40.font-change-30.font-roboto.font-weight-bold:nth-of-type(1)').get_text(strip=True) if soup.select_one('.fsize-40.font-change-30.font-roboto.font-weight-bold:nth-of-type(1)') else '情報なし'
    
    # 残業時間情報を含むセクションを探す
    overtime_section = soup.find('div', class_='col-6 px-0 text-center pt-1')
    if overtime_section:
        # 残業時間の値を含むspan要素を探す
        overtime_span = overtime_section.find('span', class_='font-change-30')
        overtime = overtime_span.get_text(strip=True) if overtime_span else '情報なし'
    else:
        overtime = '情報なし'

    return {
        'salary': salary,
        'overtime': overtime
    }


In [32]:
def fetch_salary_details(company_id):
    details_url = f'https://en-hyouban.com/company/{company_id}/salary/'
    response = requests.get(details_url)
    if response.ok:
        return response.text
    return None

def parse_salary_details(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    salary_details = {}
    rows = soup.select('#salary_information_by_age tr')
    for row in rows:
        age_group = row.select_one('.job-type')
        if age_group:
            age_text = age_group.get_text(strip=True)
            # Replace '以上' with '_up' and other age ranges with '_' for consistency
            age_key = age_text.replace('~', '_').replace('歳', '').replace('以上', '_up')
            salary_key = f'salary_{age_key}'  # Create a key like 'salary_25_29'
            salary_value = row.select_one('.salary-font-size').get_text(strip=True) if row.select_one('.salary-font-size') else '情報なし'
            salary_details[salary_key] = salary_value
    return salary_details




def fetch_and_parse_company_data(company):
    search_page_content = fetch_company_data(company)
    if search_page_content:
        profile_page_url = find_company_profile_page(search_page_content, company)
        if profile_page_url:
            sleep(1)  # URLを見つけた後にサーバーの負荷を減らすために待機
            profile_page_content = fetch_company_profile_page(profile_page_url)
            if profile_page_content:
                company_data = parse_company_profile_page(profile_page_content)
                # Extract company ID from profile page URL
                company_id = profile_page_url.split('/')[-2]
                salary_details_content = fetch_salary_details(company_id)
                if salary_details_content:
                    salary_details = parse_salary_details(salary_details_content)
                    company_data.update(salary_details)
                return company, company_data
    return company, None


# 結果をCSVに保存する関数 (更新されたフィールド名)
def save_results_to_csv(results, filename=fr'C:\Users\100ca\Documents\PyCode\scraping\data\results.csv'):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['company', 'salary', 'overtime', 'salary_25_29', 'salary_30_34', 'salary_35_39', 'salary_40_44', 'salary_45_49', 'salary_50_54', 'salary_55_59', 'salary_60_up']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for company, data in results.items():
            if data:
                writer.writerow({**{'company': company}, **data})

# 並列処理で各企業に対してスクレイピングを実行
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    future_to_company = {executor.submit(fetch_and_parse_company_data, company): company for company in companies}
    results = {}
    for future in concurrent.futures.as_completed(future_to_company):
        company = future_to_company[future]
        try:
            data = future.result()
            results[data[0]] = data[1]
        except Exception as exc:
            print(f'{company} generated an exception: {exc}')

# 結果をCSVに保存
save_results_to_csv(results)


# 結果の出力 (更新されたフィールド名)
for company, data in results.items():
    if data:
        print(f"{company}: 年収={data.get('salary', '情報なし')}, 残業時間={data.get('overtime', '情報なし')}, " +
              f"25-29歳平均年収={data.get('salary_25_29', '情報なし')}, 30-34歳平均年収={data.get('salary_30_34', '情報なし')}, " +
              f"35-39歳平均年収={data.get('salary_35_39', '情報なし')}, 40-44歳平均年収={data.get('salary_40_44', '情報なし')}, " +
              f"45-49歳平均年収={data.get('salary_45_49', '情報なし')}, 50-54歳平均年収={data.get('salary_50_54', '情報なし')}, " +
              f"55-59歳平均年収={data.get('salary_55_59', '情報なし')}, 60歳以上平均年収={data.get('salary_60_up', '情報なし')}")

ゴールドマン・サックス証券: 年収=1497, 残業時間=66, 25-29歳平均年収=1183, 30-34歳平均年収=1739, 35-39歳平均年収=情報なし, 40-44歳平均年収=情報なし, 45-49歳平均年収=1270, 50-54歳平均年収=情報なし, 55-59歳平均年収=情報なし, 60歳以上平均年収=情報なし
JPモルガン証券: 年収=1147, 残業時間=61, 25-29歳平均年収=情報なし, 30-34歳平均年収=情報なし, 35-39歳平均年収=情報なし, 40-44歳平均年収=800, 45-49歳平均年収=情報なし, 50-54歳平均年収=情報なし, 55-59歳平均年収=情報なし, 60歳以上平均年収=情報なし
東京海上日動火災保険: 年収=651, 残業時間=31, 25-29歳平均年収=544, 30-34歳平均年収=638, 35-39歳平均年収=651, 40-44歳平均年収=779, 45-49歳平均年収=764, 50-54歳平均年収=1051, 55-59歳平均年収=情報なし, 60歳以上平均年収=情報なし
日本政策投資銀行: 年収=861, 残業時間=35, 25-29歳平均年収=592, 30-34歳平均年収=情報なし, 35-39歳平均年収=情報なし, 40-44歳平均年収=情報なし, 45-49歳平均年収=情報なし, 50-54歳平均年収=情報なし, 55-59歳平均年収=情報なし, 60歳以上平均年収=情報なし
UBS証券: 年収=---, 残業時間=--, 25-29歳平均年収=情報なし, 30-34歳平均年収=情報なし, 35-39歳平均年収=情報なし, 40-44歳平均年収=情報なし, 45-49歳平均年収=情報なし, 50-54歳平均年収=情報なし, 55-59歳平均年収=情報なし, 60歳以上平均年収=情報なし
野村證券: 年収=769, 残業時間=32, 25-29歳平均年収=616, 30-34歳平均年収=731, 35-39歳平均年収=1042, 40-44歳平均年収=942, 45-49歳平均年収=情報なし, 50-54歳平均年収=情報なし, 55-59歳平均年収=情報なし, 60歳以上平均年収=情報なし
三菱UFJ銀行: 年収=780, 残業時間=30, 25-29歳平均年収=518, 